In [34]:
import pandas as pd

df = pd.read_csv("dataset_19.csv")

In [35]:
import random

def replace_nan(row, col_name):
    if pd.isna(row[col_name]):
        return random.choice(df[col_name].dropna().tolist())
    return row[col_name]

df['tipo_hd'] = df.apply(replace_nan, col_name='tipo_hd', axis=1)
df['tipo_processador'] = df.apply(replace_nan, col_name='tipo_processador', axis=1)
df['armazenamento_tb'] = df.apply(replace_nan, col_name='armazenamento_tb', axis=1)


df = df.dropna()

In [36]:
# change armazenamento_tb to int: 0.256 is 1, 0.512 is 2, 1.0 is 4, 2.0 is 8
df['armazenamento_tb'] = df['armazenamento_tb'].replace({0.256: 1, 0.512: 2, 1.0: 4, 2.0: 8}).astype(int)
df['tipo_hd'] = df['tipo_hd'].replace({'SSD': 1, 'HDD': 0}).astype(int)

# todo mac usa apple sillicon
# se nao eh mac pode ser linux ou nao
# se nao eh mac pode ser amd ou nao
df['is_amd'] = df['tipo_processador'].apply(lambda x: 1 if 'AMD' in x else 0)
df['is_windows'] = df['sistema_operacional'].apply(lambda x: 1 if 'Windows' in x else 0)
df['is_mac'] = df['sistema_operacional'].apply(lambda x: 1 if 'MacOS' in x else 0)
df = df.drop(columns=['tipo_processador'])
df = df.drop(columns=['sistema_operacional'])

/tmp/ipykernel_9985/3217461436.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['tipo_hd'] = df['tipo_hd'].replace({'SSD': 1, 'HDD': 0}).astype(int)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
import statsmodels.api as sm

X = df.drop(columns=['tempo_resposta'])
Y = df['tempo_resposta']

lr = LinearRegression()

selection_results = {}

for k in range(1, 8):
    sfs = SequentialFeatureSelector(
        estimator=lr,
        n_features_to_select=k,
        direction='forward', 
        scoring='r2',        
        cv=5,                
        n_jobs=-1            
    )
    sfs.fit(X, Y)
    
    mask = sfs.get_support()
    selected_feats = list(X.columns[mask])
    
    cv_scores = cross_val_score(lr, X[selected_feats], Y, cv=5, scoring='r2', n_jobs=-1)
    
    selection_results[k] = {
        'features': selected_feats,
        'mean_cv_r2': cv_scores.mean(),
        'std_cv_r2': cv_scores.std()
    }


ols_results = {}

for k, res in selection_results.items():
    feats = res['features']
    
    X_k = sm.add_constant(X[feats])
    
    model = sm.OLS(Y, X_k).fit()
    
    ols_results[k] = {
        'features': feats,
        'rsquared': model.rsquared,
        'rsquared_adj': model.rsquared_adj,
        'f_stat': model.fvalue
    }

# print a comparison table
print(f"{'k':>2}  {'R²':>7}  {'Adj. R²':>9}  {'F-stat':>9}   Selected features")
print("-"*60)
for k in sorted(ols_results):
    stats = ols_results[k]
    feats = ", ".join(stats['features'])
    print(f"{k:2d}  {stats['rsquared']:7.3f}  {stats['rsquared_adj']:9.3f}  {stats['f_stat']:9.3f}   {feats}")



 k       R²    Adj. R²     F-stat   Selected features
------------------------------------------------------------
 1    0.550      0.547    217.564   cpu_cores
 2    0.714      0.711    220.768   cpu_cores, ram_gb
 3    0.714      0.710    146.815   cpu_cores, ram_gb, tipo_hd
 4    0.715      0.709    109.972   cpu_cores, ram_gb, armazenamento_tb, tipo_hd
 5    0.715      0.707     87.476   cpu_cores, ram_gb, armazenamento_tb, tipo_hd, is_mac
 6    0.720      0.710     74.019   cpu_cores, ram_gb, armazenamento_tb, tipo_hd, is_windows, is_mac
 7    0.721      0.710     63.578   cpu_cores, ram_gb, latencia_ms, armazenamento_tb, tipo_hd, is_windows, is_mac
